In [1]:
from compare_2_courses.scrape.utils import get_default_chrome_driver
from compare_2_courses.scrape.scraper_config import ScraperConfig
from compare_2_courses.scrape.udemy_scraper import UdemyScraper, Course
from compare_2_courses.constants import (
    AWS_DEA_01_2024_HANDS_ON
    , ULTIMATE_AWS_SAA_C03
    , ULTIMATE_AWS_SAP_C02_2024
    , AWS_2024_DOP_C02
    , ULTIMATE_AWS_NEW_2024_SCS_C02
    )
import json

In [2]:
driver = get_default_chrome_driver()

In [5]:
for course in [
    AWS_DEA_01_2024_HANDS_ON
    , ULTIMATE_AWS_SAA_C03
    , ULTIMATE_AWS_SAP_C02_2024
    , AWS_2024_DOP_C02
    , ULTIMATE_AWS_NEW_2024_SCS_C02
]:
    print(course.title)
    scrape_config = ScraperConfig(course_landing_page=course.landing_page)
    scraper = UdemyScraper(scraper_config=scrape_config, driver=driver)
    course = scraper.scrape()
    with open(f"./data/{course.title}.json", "w+") as f:
        json.dump(course.to_json_dict(),f)

AWS Certified Data Engineer Associate 2024 - Hands On!
Ultimate AWS Certified Solutions Architect Associate SAA-C03
Ultimate AWS Certified Solutions Architect Professional 2024
AWS Certified DevOps Engineer Professional 2024 - DOP-C02
Ultimate AWS Certified Security Specialty [NEW 2024] SCS-C02
